In [16]:
using NCDatasets
using Dates
using Glob
using DIVAnd
using Test
using CSV
using DataFrames

In [6]:
doifile = "../../diva_2023_doi.csv"

"../../diva_2023_doi.csv"

In [13]:
# User inputs
# ------------

hostname = gethostname()
if hostname == "ogs04"
	@info "Working in production server"
	databasedir = "/production/apache/data/emodnet-domains/By sea regions/Black Sea"
elseif hostname == "GHER-ULg-Laptop"
	outputbasedir = "/data/EMODnet/Chemistry/merged/"
	databasedir = "/data/EMODnet/Chemistry/prod/"
elseif hostname == "gherdivand"
	outputbasedir = "/home/ctroupin/data/EMODnet/merged"
	databasedir = "/home/ctroupin/data/EMODnet/By sea regions"
elseif hostname == "FSC-PHYS-GHER01"
    outputbasedir = "/home/ctroupin/data/EMODnet-Chemistry/merged"
	databasedir = "/home/ctroupin/data/EMODnet-Chemistry/emodnet-results-2023"
else
	@error("Unknown host")
end

"/home/ctroupin/data/EMODnet-Chemistry/emodnet-results-2023"

In [54]:
function get_ncfile_list(datadir::String)::Array
    filelist = []
	# Varname with spaces instead of underscores
	for (root, dirs, files) in walkdir(datadir)
        for file in files
            if endswith(file, ".nc")
                push!(filelist, joinpath(root, file))
            end
        end
    end
    @info("Found $(length(filelist)) files")
    return filelist
end

get_ncfile_list (generic function with 1 method)

In [55]:
# Read the CSV
data = CSV.read(doifile, DataFrame, header=["productID", "productTitle", "productDOI"]);

In [56]:
# Generate list of netCDF
resultfilelist = get_ncfile_list(databasedir);

[ Info: Found 53 files


In [57]:
for resfile in resultfilelist
    @debug("Working on file $(resfile)")
    
    NCDataset(resfile, "a") do ds
        productID = ds.attrib["product_id"]
        #@info(productID);
        
        productIndex = findfirst(data.productID .== productID)
        @info(productIndex);
        
        if productIndex == nothing
            @info(resfile);
        else
            @info("Modify the DOI")
            oldDOI = ds.attrib["doi"]
            newDOI = data.productDOI[productIndex]
            @info("$(oldDOI) → $(newDOI)")
        end
    end
end

[ Info: nothing
[ Info: /home/ctroupin/data/EMODnet-Chemistry/emodnet-results-2023/All_European_Seas/Water_body_ammonium.nc
[ Info: nothing
[ Info: /home/ctroupin/data/EMODnet-Chemistry/emodnet-results-2023/All_European_Seas/Water_body_chlorophyll-a.nc
[ Info: nothing
[ Info: /home/ctroupin/data/EMODnet-Chemistry/emodnet-results-2023/All_European_Seas/Water_body_dissolved_inorganic_nitrogen.nc
[ Info: nothing
[ Info: /home/ctroupin/data/EMODnet-Chemistry/emodnet-results-2023/All_European_Seas/Water_body_dissolved_oxygen_concentration.nc
[ Info: nothing
[ Info: /home/ctroupin/data/EMODnet-Chemistry/emodnet-results-2023/All_European_Seas/Water_body_phosphate.nc
[ Info: nothing
[ Info: /home/ctroupin/data/EMODnet-Chemistry/emodnet-results-2023/All_European_Seas/Water_body_silicate.nc
[ Info: 1
[ Info: Modify the DOI
[ Info: 10.6092/30f5c819-bb30-4dee-9afd-0df48682495a → https://doi.org/10.13120/30f5c819-bb30-4dee-9afd-0df48682495a
[ Info: 2
[ Info: Modify the DOI
[ Info: 10.6092/e871c976-

[ Info: 10.6092/a435e06d-12cd-4056-b2cd-7cf6d2a68d67 → https://doi.org/10.13120/13588424-5d4d-43c6-8b04-4c871c864c0e
[ Info: 16
[ Info: Modify the DOI
[ Info: 10.6092/fda28b2d-86ed-4aa9-b1ea-a3bffde379fb → https://doi.org/10.13120/fda28b2d-86ed-4aa9-b1ea-a3bffde379fb
[ Info: 17
[ Info: Modify the DOI
[ Info: 10.6092/41931b71-2ed6-4a25-8712-368ae1d904da → https://doi.org/10.13120/41931b71-2ed6-4a25-8712-368ae1d904da
[ Info: 18
[ Info: Modify the DOI
[ Info: 10.6092/1c1e5e79-3d6f-4998-b7cf-59e3e148ddbc → https://doi.org/10.13120/8f127592-cff5-4eb3-b3a7-c66edcaf9473
[ Info: 29
[ Info: Modify the DOI
[ Info: 10.6092/61e77250-b68f-11ed-1190-4bfda496df09 → https://doi.org/10.13120/61e77250-b68f-11ed-1190-4bfda496df09
[ Info: 30
[ Info: Modify the DOI
[ Info: 10.6092/a5767700-b6df-11ed-12fe-a98c192982d1 → https://doi.org/10.13120/a5767700-b6df-11ed-12fe-a98c192982d1
[ Info: 31
[ Info: Modify the DOI
[ Info: 10.6092/2c896f10-b61b-11ed-34f9-d3cbc75d1d06 → https://doi.org/10.13120/2c896f10-b61b-

In [47]:
split(data.productTitle[1], "-")[1]

"Arctic Ocean "

In [51]:
data.productDOI[15]

"https://doi.org/10.13120/13588424-5d4d-43c6-8b04-4c871c864c0e"